<a href="https://colab.research.google.com/github/berilldindar/OkCupidProfilesTextMining/blob/main/OKCupid_ModelingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/drive')
%cd /drive

Mounted at /drive
/drive


In [ ]:
import os
os.chdir('/drive/My Drive/Text Mining')

In [ ]:
# Makine öğrenmesi ve veri analizi için kullanacağımız kütüphaneleri yükleriz.
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

sns.set_style("whitegrid")
sns.set_context("poster")

In [ ]:
# Kullanım için .csv'yi veri çerçevesine yükledik
df = pd.read_csv('new_df.csv')
df = df.drop('Unnamed: 0', 1)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,pets,religion,sign,smokes,speaks,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75.0,-1.0,transportation,2012-06-28-20-30,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes,english,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,70.0,NaN,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (...",i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,68.0,-1.0,NaN,2012-06-27-09-10,"san francisco, california",NaN,has cats,NaN,pisces but it doesn&rsquo;t matter,no,"english, french, c++","i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,71.0,20000.0,student,2012-06-28-14-22,"berkeley, california",doesn't want kids,likes cats,NaN,pisces,no,"english, german (poorly)",i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",66.0,-1.0,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",NaN,likes dogs and likes cats,NaN,aquarius,no,english,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


In [ ]:
# Makine Öğrenimi için olası değişkenleri belirttik bunlar cinsiyet essayler,vücut tipleri,alkol,uyuşturucu kullanım,eğitim,etnik köken, gelir
n_men = len(df[df.sex == 'm'])
n_women = len(df[df.sex == 'f'])
n_essays = (df.loc[:,'essay0':'essay9']).size
n_body = df.body_type.unique().size
n_drink = df.drinks.unique().size
n_drug = df.drugs.unique().size
n_education = df.education.unique().size
n_ethnicity = df.ethnicity.unique().size
n_income = df.income.unique().size
n_orientation = df.orientation.unique().size

print("Number of male users: {:d}".format(n_men))
print("Number of essays: {:d}".format(n_essays))
print("Number of female users: {:d}".format(n_women))
print("Number of body types: {:d}".format(n_body))
print("Number of drinking types: {:d}".format(n_drink))
print("Number of drug types: {:d}".format(n_drug))
print("Number of ethnicity combinations: {:d}".format(n_ethnicity))
print("Number of income levels: {:d}".format(n_income))
print("Number of orientation types: {:d}".format(n_orientation))

Number of male users: 35829
Number of essays: 599460
Number of female users: 24117
Number of body types: 13
Number of drinking types: 7
Number of drug types: 4
Number of ethnicity combinations: 218
Number of income levels: 14
Number of orientation types: 3


In [ ]:
# Her kullanıcı için 10 essayin tümünü tek bir sütunda birleştirdik
# Ayrıca eksik essayleri boş değerlerle değiştirdik 
df['all_essays'] = ''
essay_names = df.loc[:,'essay0':'essay9']
for essay_name in essay_names:
    df[essay_name] = df[essay_name].replace(np.nan, ' ')
    df['all_essays'] = df[essay_name] + ' ' + df['all_essays']

In [ ]:
# Essaylerden cinsiyet tahminine yönelik veri çerçevesi
# Diğer değişkenleri dahil etmek için yeniden üretilebilir
essay_sex = df[['all_essays', 'sex']]
essay_sex.head(15)

,all_essays,sex
0,you want to be swept off your feet! you are ti...,m
1,i am very open and will share just about any...,m
2,"you are bright, open, intense, silly, ironic, ...",m
3,you feel so inclined. cats and german phil...,m
4,"music: bands, rappers, musicians at ...",m
5,you're awesome. i cried on my first day at sch...,m
6,my typical friday night plotting to take ove...,f
7,out and about or relaxing at home with a g...,f
8,http://www.youtube.com/watch?v=4dxbwzuwsxk let...,f
9,you can rock the bells and say hi. send a mess...,m


In [ ]:
# Essayleri bir vektör uzay modeline dönüştürdük (Cinsiyete göre dönüştürdük ve erkeklere göre integer olarak dönüştürdük)
from sklearn.feature_extraction.text import CountVectorizer

def make_xy(essay_sex, vectorizer=None):   
    if vectorizer is None:
        vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(essay_sex.all_essays)
    X = X.tocsc() 
    y = (essay_sex.sex == 'm').values.astype(np.int)
    return X, y
X, y = make_xy(essay_sex)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#Modelimizi Multinomial Naive Bayes eğittik eğiteceğimiz x ve y olarak ayırdık ve split ettik
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y)
clf = MultinomialNB().fit(xtrain, ytrain)
print("MN Accuracy: %0.2f%%" % (100 * clf.score(xtest, ytest)))

MN Accuracy: 76.61%


In [ ]:
# Eğitilenin doğruluk ve test skoru
training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: %0.2f" % (training_accuracy))
print("Accuracy on test data:     %0.2f" % (test_accuracy))

Accuracy on training data: 0.84
Accuracy on test data:     0.77


In [ ]:
#Confusion matrixi çizdirdik
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest, clf.predict(xtest)))

[[4518 1430]
 [2075 6964]]


In [ ]:
#K-Folds Cross Validation’da verilerimizi k farklı alt kümeye böleriz. Verilerimizi eğitmek ve son alt kümeyi test verisi olarak bırakmak için k-1 adet alt kümeyi kullanırız. 
# k adet deney sonucunda ortaya çıkan ortalama hata değeri modelimizin geçerliliğini belirtir.
from sklearn.model_selection import KFold
def cv_score(clf, X, y, scorefunc):
    result = 0.
    nfold = 5
    for train, test in KFold(nfold).split(X): # verileri 5 kez train/test gruplarına ayırın
        clf.fit(X[train], y[train]) 
        result += scorefunc(clf, X[test], y[test]) 
    return result / nfold #ortalamasını aldık

In [ ]:
#logaritmasını aldık erkek ve kadınlar için olasılığını hesaplarız
def log_likelihood(clf, x, y):
    prob = clf.predict_log_proba(x)
    female = y == 0
    male = ~female
    return prob[female, 0].sum() + prob[male, 1].sum()

In [ ]:
from sklearn.model_selection import train_test_split
_, itest = train_test_split(range(essay_sex.shape[0]), train_size=0.7)
mask = np.zeros(essay_sex.shape[0], dtype=np.bool)
mask[itest] = True

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#üzerinde arama yapılacak parametreler
alphas = [.1, 1, 5, 10, 50] #alpha değerlerimiz 
min_dfs = [1e-5, 1e-4, 1e-3, 1e-2] #minimum dfs değerleri

#alpha ve min_df için en iyi değeri ve en iyi sınıflandırıcıyı bulmak için
best_alpha = None
best_min_df = None
maxscore=-np.inf
for alpha in alphas:
    for min_df in min_dfs:         
        vectorizer = CountVectorizer(min_df = min_df)       
        Xthis, ythis = make_xy(essay_sex, vectorizer)
        Xtrainthis=Xthis[mask]
        ytrainthis=ythis[mask]
        clf = MultinomialNB(alpha=alpha)
        cvscore = cv_score(clf, Xtrainthis, ytrainthis, log_likelihood)

        if cvscore > maxscore:
            maxscore = cvscore
            best_alpha, best_min_df = alpha, min_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

In [ ]:
print('alpha: {:.2f}'.format(best_alpha))
print('min_df: {}'.format(best_min_df))

alpha: 50.00
min_df: 0.01


In [ ]:
vectorizer = CountVectorizer(min_df=best_min_df)
X, y = make_xy(essay_sex, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]

clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)

training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: %0.2f" % (training_accuracy))
print("Accuracy on test data:     %0.2f" % (test_accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


Accuracy on training data: 0.76
Accuracy on test data:     0.74


In [ ]:
# Cinsiyeti tahmin eden kelimelerin listesini aldık 
words = np.array(vectorizer.get_feature_names()) #özelliklerin isimlerini vektörize ederiz

x = np.eye(xtest.shape[1])
probs = clf.predict_log_proba(x)[:, 0]
ind = np.argsort(probs)

men_words = words[ind[:10]]
women_words = words[ind[-10:]]

men_prob = probs[ind[:10]]
women_prob = probs[ind[-10:]]

print("Men words\t           P(Men word | word)")
for w, p in zip(men_words, men_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))
    
print("Women words\t        P(Men word | word)")
for w, p in zip(women_words, women_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

#Kadınlar ve erkeklerin essaylerde bahsettiği ilgili kelimelerin olasılıklarını sıraladık

Men words	           P(Men word | word)
           computers 0.85
            software 0.82
                 guy 0.80
            engineer 0.80
               beard 0.80
         engineering 0.80
              fixing 0.79
             startup 0.77
              guitar 0.77
         programming 0.77
Women words	        P(Men word | word)
               curly 0.38
               queer 0.38
              baking 0.35
            fabulous 0.35
               adore 0.34
           chapstick 0.34
           prejudice 0.33
                 gal 0.33
             anatomy 0.32
               heels 0.27


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
x, y = make_xy(essay_sex, vectorizer)

prob = clf.predict_proba(x)[:, 0]
predict = clf.predict(x)

bad_women = np.argsort(prob[y == 0])[:3]
bad_men = np.argsort(prob[y == 1])[-3:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
print("Yanlış Tahmin Edilen Kadınlardan Alıntılar")
print('---------------------------')
for row in bad_women:
    print(essay_sex[y == 0].all_essays.iloc[row])
    print('')

Mis-predicted Women quotes
---------------------------
... you're someone who could get down with just stopping by for a cup of tea and a dance sometimes, not much conversation really, maybe a bowl, maybe not.  ... you'd go out with me on halloween dressed up as an oompa-loompa riding a horsey-on-a-stick... i'll dress as one too.  ... or... or... i could dress up as the oompa-loompa riding the horsey and, oh my god... wai-wait... this is great... you could be the guy with the broom and dustpan that trails us--me and the horsey--and sweeps up our droppings... party droppings, you know, chewed gum, cherry stems, shoe heels, shell casings, torsos with the heads chewed off, like at a parade, yes! we'll be a parade... me and you and a horsey-on-a-stick... a broom and a dustpan and a bodybag perhaps... your tools will be all spangly, and your mustache like a whiskbroom atop your lip... of course you'll party with us, and we'll wait for you to finish cleaning up wherever we stop...  whoa... t

In [ ]:
print("Yanlış Tahmin Edilen Erkeklerden Alıntılar")
print('--------------------------')
for row in bad_men:
    print(essay_sex[y == 1].all_essays.iloc[row])
    print('')

Mis-predicted Men quotes
--------------------------
if you want to.  for any or all of the inspirations mentioned below. or others.  you feel. and love feeling as a primary mode of being & relating.  one basic requirement is the expression of at least a basic level of integrity in your way relating with people ( of which i am one, of course ) by communicating honestly & responsively with people whether you are personally interested in them or not. expressing interest in meeting with someone, or meeting then simply ignoring & not communicating honestly & timely with them about meeting again or to their communications is a strangely disconnected, unintegral way of being that seems to be too common in this society.  i make a point of respecting people equally to myself even when i am not personally interested in relating with them again, by at least respecting their equal humanity, clarity of energy i contribute to the world & my own integrity enough to respond honestly & timely to offers

In [ ]:
text = ['I am a programmer with a beard']

vectorizer = CountVectorizer(min_df=best_min_df,
                             vocabulary=vectorizer.get_feature_names())
x = vectorizer.fit_transform(text)
prob = clf.predict_proba(vectorizer.transform(text))
man_prob = prob[0, 1]
woman_prob = prob[0, 0]
man_prob = ('%.2f' % (100*man_prob))
woman_prob = ('%.2f' % (100*woman_prob))
# Prediction
print("Text/Essay: ", text)
print('')
if clf.predict(x) == 1:
    print("This is", man_prob, "percent likely from a man's essay")
else:
    print("This is", woman_prob, "percent likely from a woman's essay")

Text/Essay:  ['I am a programmer with a beard']

This is 74.94 percent likely from a man's essay


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
text = ['I like lipgloss and lipstick']

vectorizer = CountVectorizer(min_df=best_min_df,
                             vocabulary=vectorizer.get_feature_names())
x = vectorizer.fit_transform(text)
prob = clf.predict_proba(vectorizer.transform(text))
man_prob = prob[0, 1]
woman_prob = prob[0, 0]
man_prob = ('%.2f' % (100*man_prob))
woman_prob = ('%.2f' % (100*woman_prob))
# Prediction
print("Text/Essay: ", text)
print('')
if clf.predict(x) == 1:
    print("This is", man_prob, "percent likely from a man's essay")
else:
    print("This is", woman_prob, "percent likely from a woman's essay")


Text/Essay:  ['I like lipgloss and lipstick']

This is 59.75 percent likely from a man's essay


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
text = ['I like computers']
vectorizer = CountVectorizer(min_df=best_min_df,
                             vocabulary=vectorizer.get_feature_names())
x = vectorizer.fit_transform(text)
prob = clf.predict_proba(vectorizer.transform(text))
man_prob = prob[0, 1]
woman_prob = prob[0, 0]
man_prob = ('%.2f' % (100*man_prob))
woman_prob = ('%.2f' % (100*woman_prob))
# Prediction
print("Text/Essay: ", text)
print('')
if clf.predict(x) == 1:
    print("This is", man_prob, "percent likely from a man's essay")
else:
    print("This is", woman_prob, "percent likely from a woman's essay")

Text/Essay:  ['I like computers']

This is 85.87 percent likely from a man's essay


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
text = ['I hate computers']
vectorizer = CountVectorizer(min_df=best_min_df,
                             vocabulary=vectorizer.get_feature_names())
x = vectorizer.fit_transform(text)
prob = clf.predict_proba(vectorizer.transform(text))
man_prob = prob[0, 1]
woman_prob = prob[0, 0]
man_prob = ('%.2f' % (100*man_prob))
woman_prob = ('%.2f' % (100*woman_prob))
# Prediction
print("Text/Essay: ", text)
print('')
if clf.predict(x) == 1:
    print("This is", man_prob, "percent likely from a man's essay")
else:
    print("This is", woman_prob, "percent likely from a woman's essay")

Text/Essay:  ['I hate computers']

This is 83.32 percent likely from a man's essay


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
text = ['hate']
vectorizer = CountVectorizer(min_df=best_min_df,
                             vocabulary=vectorizer.get_feature_names())
x = vectorizer.fit_transform(text)
prob = clf.predict_proba(vectorizer.transform(text))
man_prob = prob[0, 1]
woman_prob = prob[0, 0]
man_prob = ('%.2f' % (100*man_prob))
woman_prob = ('%.2f' % (100*woman_prob))
# Prediction
print("Text/Essay: ", text)
print('')
if clf.predict(x) == 1:
    print("This is", man_prob, "percent likely from a man's essay")
else:
    print("This is", woman_prob, "percent likely from a woman's essay")

Text/Essay:  ['hate']

This is 56.54 percent likely from a man's essay


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
text = ['like']
vectorizer = CountVectorizer(min_df=best_min_df,
                             vocabulary=vectorizer.get_feature_names())
x = vectorizer.fit_transform(text)
prob = clf.predict_proba(vectorizer.transform(text))
man_prob = prob[0, 1]
woman_prob = prob[0, 0]
man_prob = ('%.2f' % (100*man_prob))
woman_prob = ('%.2f' % (100*woman_prob))
# Prediction
print("Text/Essay: ", text)
print('')
if clf.predict(x) == 1:
    print("This is", man_prob, "percent likely from a man's essay")
else:
    print("This is", woman_prob, "percent likely from a woman's essay")

Text/Essay:  ['like']

This is 61.28 percent likely from a man's essay


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2),min_df=best_min_df)
X, y = make_xy(essay_sex, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]

clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)

training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

#ngramları kullanıp tekrar vektörleştirdik

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


Accuracy on training data: 0.750723
Accuracy on test data:     0.732282


In [ ]:
vectorizer = CountVectorizer(ngram_range=(2,2),min_df=best_min_df)
X, y = make_xy(essay_sex, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]

clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)

training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


Accuracy on training data: 0.719417
Accuracy on test data:     0.695868


In [ ]:
from sklearn.ensemble import RandomForestClassifier

vectorizer = CountVectorizer(min_df=best_min_df)
X, y = make_xy(essay_sex, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]

rforest = RandomForestClassifier()
clf = rforest.fit(xtrain,ytrain)

training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


Accuracy on training data: 0.987100
Accuracy on test data:     0.730256


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=best_min_df)
X, y = make_xy(essay_sex, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]

clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)

training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


Accuracy on training data: 0.595140
Accuracy on test data:     0.598851


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=best_min_df)
X, y = make_xy(essay_sex, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]

clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)

training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


Accuracy on training data: 0.595029
Accuracy on test data:     0.598828
